In [6]:
import numpy as np
import pandas as pd
import pdb

In [7]:
pwd

'/Users/mingyuanma/Desktop/hai-cooking/analysis new'

In [8]:
s1 = pd.read_csv("../data/Archive/combined_phase2_s1.csv")
t1 = pd.read_csv("../data/Archive/combined_phase2_t1.csv")
control = pd.read_csv("../data/Archive/combined_phase2.csv")


In [ ]:
# #     df = df.rename(columns={0:"idle_2", 1:"chop", 2:"cook", 3:"plate"})

# #     df = df.join(pd.get_dummies(df["workerId"]))
# #     warmup = df[df["round"] <= 2]
# #     warmup = warmup.rename(columns={0:"idle", 1:"chef", 2:"souchef", 3:"server"})
# #     start = df[df["round"] >= 3]
# # #     start = start.rename(columns={0:"idle", 1:"chef", 2:"server", 3:"souchef"})
# #     start = start.rename(columns={0:"idle", 1:"souchef", 2:"server", 3:"chef"})

# Sou-Chef Chopping 0.02 0.02 -0.31 -0.28 -0.16 -0.21
# Sou-Chef Cooking 0.00 0.00 -0.08 0.28 0.39 0.22
# Sou-Chef Plating 0.03 -0.05 -0.25 -0.30 -0.38 -0.24
# Chef Chopping -0.13 -0.09 — — — —
# Chef Cooking -0.15 -0.09 — — — —
# Chef Plating 0.30 -0.02 — — — —
# Server Chopping 0.05 0.07 -0.01 0.08 0.03 0.04
# Server Cooking 0.00 -0.09 0.00 0.96 -0.06 0.00
# Server Plating

### round-scale analysis

In [16]:
import scipy

In [17]:
pd.set_option('mode.chained_assignment', None)

In [97]:
def get_rounds(df):
    df = df.copy()
    warmup = df[df["round"] <= 2]
    warmup["Sou-Chef Chopping"] = (warmup["workerId"]==2) & (warmup["taskId"]==1)
    warmup["Sou-Chef Cooking"] = (warmup["workerId"]==2) & (warmup["taskId"]==2)
    warmup["Sou-Chef Plating"] = (warmup["workerId"]==2) & (warmup["taskId"]==3)
    warmup["Chef Chopping"] = (warmup["workerId"]==1) & (warmup["taskId"]==1)
    warmup["Chef Cooking"] = (warmup["workerId"]==1) & (warmup["taskId"]==2)
    warmup["Chef Plating"] = (warmup["workerId"]==1) & (warmup["taskId"]==3)
    warmup["Server Chopping"] =(warmup["workerId"]==3) & (warmup["taskId"]==1)
    warmup["Server Cooking"] = (warmup["workerId"]==3) & (warmup["taskId"]==2)
    warmup["Server Plating"] = (warmup["workerId"]==3) & (warmup["taskId"]==3)
    warmup = warmup.iloc[:, [0, 1, 9, 10, 11, 12, 13, 14, 15, 16, 17]]
    warmup = warmup.groupby(["ResponseId", "round"]).sum()
    warmup = warmup.reset_index()
    round1 = warmup[warmup['round']==1]
    round2 = warmup[warmup['round']==2]

    start = df[df["round"] > 2]
    start["Sou-Chef Chopping"] = (start["workerId"]==1) & (start["taskId"]==1)
    start["Sou-Chef Cooking"] = (start["workerId"]==1) & (start["taskId"]==2)
    start["Sou-Chef Plating"] = (start["workerId"]==1) & (start["taskId"]==3)
    start["Server Chopping"] =(start["workerId"]==2) & (start["taskId"]==1)
    start["Server Cooking"] = (start["workerId"]==2) & (start["taskId"]==2)
    start["Server Plating"] = (start["workerId"]==2) & (start["taskId"]==3)

    start = start.iloc[:, [0, 1, 9, 10, 11, 12, 13, 14]]
    start = start.groupby(["ResponseId", "round"]).sum()
    start = start.reset_index()
    round3 = start[start['round']==3]
    round4 = start[start['round']==4]
    round5 = start[start['round']==5]
    round6 = start[start['round']==6]
    rounds = [round1, round2, round3, round4, round5, round6]
    return rounds

### wilcoxin

In [123]:
def wilcoxin_round(df1, df2):
    final_record = []
    control_rounds = get_rounds(df1)
    s1_rounds = get_rounds(df2)
    feature_index = 0
    for feature_index in np.arange(9):
        df_index_fixed = feature_index+2
        cur_record = []
        for i in np.arange(6):
            if i < 2:
                df_index = df_index_fixed
                x = control_rounds[i].iloc[:, df_index]
                y = s1_rounds[i].iloc[:, df_index]
                sample_len = min(x.shape[0], y.shape[0])
                x, y = x.iloc[:sample_len, ], y.iloc[:sample_len, ]
                cur_record.append(scipy.stats.wilcoxon(
                    x, y
                )[1])     
            else:
                if feature_index in [3,4,5]:
                    cur_record.append(0)
                else:
                    if feature_index > 5:
                        df_index = df_index_fixed - 3
                    x = control_rounds[i].iloc[:, df_index]
                    y = s1_rounds[i].iloc[:, df_index]
                    sample_len = min(x.shape[0], y.shape[0])
                    x, y = x.iloc[:sample_len, ], y.iloc[:sample_len, ]
                    cur_record.append(scipy.stats.wilcoxon(
                        x, y
                    )[0])
        final_record.append(cur_record)
    df = pd.DataFrame(final_record)
    df = df.rename(columns={0:"Round 1", 1:"Round 2", 2:"Round 3", 3:"Round 4", 4:"Round 5", 5:"Round 6"})
    df = df.rename({0:"Sou-Chef Chopping", 1:"Sou-Chef Cooking", 2:"Sou-Chef Plating", 
                    3:"Chef Chopping", 4:"Chef Cooking", 5:"Chef Plating",
                   6: "Server Chopping", 7: "Server Cooking", 8:"Server Plating"}, axis=0)
    df = df.reset_index()
    print(
        df.to_latex(index=False,
                    formatters={"name": str.upper},
                    float_format="{:.3f}".format)
    )
    return df

wilcoxin_round(control, s1)

\begin{tabular}{lrrrrrr}
\toprule
index & Round 1 & Round 2 & Round 3 & Round 4 & Round 5 & Round 6 \\
\midrule
Sou-Chef Chopping & 0.100 & 0.935 & 6265.500 & 4838.500 & 3565.500 & 3697.000 \\
Sou-Chef Cooking & 0.006 & 0.106 & 4727.500 & 3007.500 & 3426.500 & 2616.500 \\
Sou-Chef Plating & 0.075 & 0.395 & 6634.000 & 4956.000 & 6025.500 & 6477.000 \\
Chef Chopping & 0.731 & 0.579 & 0.000 & 0.000 & 0.000 & 0.000 \\
Chef Cooking & 0.902 & 0.550 & 0.000 & 0.000 & 0.000 & 0.000 \\
Chef Plating & 0.444 & 0.330 & 0.000 & 0.000 & 0.000 & 0.000 \\
Server Chopping & 0.689 & 0.918 & 6790.000 & 5729.000 & 3734.500 & 3789.000 \\
Server Cooking & 0.796 & 0.080 & 2539.500 & 1500.000 & 1415.500 & 1187.000 \\
Server Plating & 0.920 & 0.085 & 8237.500 & 6866.000 & 5711.500 & 7023.500 \\
\bottomrule
\end{tabular}



,index,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6
0,Sou-Chef Chopping,0.099638,0.934704,6265.5,4838.5,3565.5,3697.0
1,Sou-Chef Cooking,0.005671,0.105619,4727.5,3007.5,3426.5,2616.5
2,Sou-Chef Plating,0.074696,0.394789,6634.0,4956.0,6025.5,6477.0
3,Chef Chopping,0.730979,0.578612,0.0,0.0,0.0,0.0
4,Chef Cooking,0.901927,0.550057,0.0,0.0,0.0,0.0
5,Chef Plating,0.443934,0.329629,0.0,0.0,0.0,0.0
6,Server Chopping,0.689090,0.917758,6790.0,5729.0,3734.5,3789.0
7,Server Cooking,0.795635,0.079902,2539.5,1500.0,1415.5,1187.0
8,Server Plating,0.920414,0.085445,8237.5,6866.0,5711.5,7023.5


### ttest

In [120]:
def ttest_round(df1, df2):
    final_record = []
    control_rounds = get_rounds(df1)
    s1_rounds = get_rounds(df2)
    feature_index = 0
    for feature_index in np.arange(9):
        df_index_fixed = feature_index+2
        cur_record = []
        for i in np.arange(6):
            if i < 2:
                df_index = df_index_fixed
                x = control_rounds[i].iloc[:, df_index]
                y = s1_rounds[i].iloc[:, df_index]
                sample_len = min(x.shape[0], y.shape[0])
                x, y = x.iloc[:sample_len, ], y.iloc[:sample_len, ]
                cur_record.append(scipy.stats.ttest_ind(
                    x, y
                )[1])     
            else:
                if feature_index in [3,4,5]:
                    cur_record.append(0)
                else:
                    if feature_index > 5:
                        df_index = df_index_fixed - 3
                    x = control_rounds[i].iloc[:, df_index]
                    y = s1_rounds[i].iloc[:, df_index]
                    sample_len = min(x.shape[0], y.shape[0])
                    x, y = x.iloc[:sample_len, ], y.iloc[:sample_len, ]
                    cur_record.append(scipy.stats.ttest_ind(
                        x, y
                    )[0])
        final_record.append(cur_record)
    df = pd.DataFrame(final_record)
    df = df.rename(columns={0:"Round 1", 1:"Round 2", 2:"Round 3", 3:"Round 4", 4:"Round 5", 5:"Round 6"})
    df = df.rename({0:"Sou-Chef Chopping", 1:"Sou-Chef Cooking", 2:"Sou-Chef Plating", 
                    3:"Chef Chopping", 4:"Chef Cooking", 5:"Chef Plating",
                   6: "Server Chopping", 7: "Server Cooking", 8:"Server Plating"}, axis=0)
    df = df.reset_index()
    print(
        df.to_latex(index=False,
                    formatters={"name": str.upper},
                    float_format="{:.3f}".format)
    )
    return df

ttest_round(control, t1)

\begin{tabular}{lrrrrrr}
\toprule
index & Round 1 & Round 2 & Round 3 & Round 4 & Round 5 & Round 6 \\
\midrule
Sou-Chef Chopping & 0.458 & 0.508 & -1.970 & -2.715 & -3.838 & -2.997 \\
Sou-Chef Cooking & 0.099 & 0.183 & -0.446 & 2.219 & 3.345 & 3.117 \\
Sou-Chef Plating & 0.352 & 0.533 & 0.125 & -3.174 & -2.576 & -3.941 \\
Chef Chopping & 0.963 & 0.676 & 0.000 & 0.000 & 0.000 & 0.000 \\
Chef Cooking & 0.602 & 0.078 & 0.000 & 0.000 & 0.000 & 0.000 \\
Chef Plating & 0.375 & 0.895 & 0.000 & 0.000 & 0.000 & 0.000 \\
Server Chopping & 0.593 & 0.344 & 1.511 & 3.084 & 2.590 & 3.335 \\
Server Cooking & 1.000 & 0.325 & -3.539 & -4.762 & -4.958 & -5.488 \\
Server Plating & 0.847 & 0.420 & -0.125 & 1.023 & 0.450 & 1.906 \\
\bottomrule
\end{tabular}



,index,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6
0,Sou-Chef Chopping,0.458366,0.508493,-1.969661,-2.715055,-3.837881,-2.996801
1,Sou-Chef Cooking,0.098559,0.183257,-0.446161,2.218828,3.345125,3.117281
2,Sou-Chef Plating,0.351938,0.532690,0.125435,-3.174116,-2.576076,-3.941487
3,Chef Chopping,0.962537,0.675720,0.000000,0.000000,0.000000,0.000000
4,Chef Cooking,0.601624,0.077533,0.000000,0.000000,0.000000,0.000000
5,Chef Plating,0.374763,0.895207,0.000000,0.000000,0.000000,0.000000
6,Server Chopping,0.593407,0.344492,1.511429,3.084146,2.589967,3.334709
7,Server Cooking,1.000000,0.324845,-3.539210,-4.761523,-4.957509,-5.488098
8,Server Plating,0.847384,0.420039,-0.125184,1.022774,0.450036,1.906067


### Tisck-scale 

In [63]:
from tqdm import tqdm
def get_rounds_ticks(df):
    df = df.copy()
    feature_pairs = [("chef_cook", "souchef_chop"), ("chef_cook", "server_plate"), ("souchef_chop", "server_cook"),
                    ("souchef_cook", "server_chop"), ("souchef_cook", "server_plate"),
                     ("souchef_plate","server_cook"), ("server_chop","souchef_cook")]


    spans = 6
    steps = spans - 1

    df = df.join(pd.get_dummies(df["taskId"]))
    df = df.rename(columns={0:"idle_2", 1:"chop", 2:"cook", 3:"plate"})

    df = df.join(pd.get_dummies(df["workerId"]))
    warmup = df[df["round"] <= 2]
    warmup = warmup.rename(columns={0:"idle", 1:"chef", 2:"souchef", 3:"server"})
    start = df[df["round"] >= 3]

    start = start.rename(columns={0:"idle", 1:"souchef", 2:"server", 3:"chef"})
    start = start.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,15]]
    df = pd.concat([warmup, start],axis=0)

    df["chef_chop"] = df["chef"] & df["chop"]
    df["chef_cook"] = df["chef"] & df["cook"]
    df["chef_plate"] = df["chef"] & df["plate"]

    df["souchef_chop"] = df["souchef"] & df["chop"]
    df["souchef_cook"] = df["souchef"] & df["cook"]
    df["souchef_plate"] = df["souchef"] & df["plate"]

    df["server_chop"] = df["server"] & df["chop"]
    df["server_cook"] = df["server"] & df["cook"]
    df["server_plate"] = df["server"] & df["plate"]

    final_dic = {}
    for rou in tqdm(np.arange(1,7)):
        cur_dic = {}
        for features in feature_pairs:
            feature_a = features[0]
            feature_b = features[1]
            feature = feature_a + "_" + feature_b
            response_array = []
            for responser in df["ResponseId"].unique():
                count = 0
                sub_df = df[df["round"] == rou]
                sub_df = sub_df[sub_df["ResponseId"] == responser]
                for t in np.sort(sub_df["tick"].unique())[:-steps]:
                    if np.sum(sub_df[sub_df["tick"] == t][feature_a]) > 0:
                        window = sub_df[(sub_df["tick"] >= t) & (sub_df["tick"] <= t+steps)]
                        if np.sum(window[feature_b]) > 0:
                            count += 1
                response_array.append(count)

            cur_dic[feature] = np.array(response_array)
        final_dic[rou] = cur_dic
    return final_dic 

control_rounds_t = get_rounds_ticks(control)
s1_rounds_t = get_rounds_ticks(s1)
t1_rounds_t = get_rounds_ticks(t1)

100%|█████████████████████████████████████████████| 6/6 [01:14<00:00, 12.49s/it]


### wilcoxin

In [77]:
def wilcoxin_ticks(group1, group2):
# group1 = s1_rounds_t
# group2 = control_rounds_t

    features = ["chef_cook_souchef_chop",
                "chef_cook_server_plate",
                "souchef_chop_server_cook",
                "souchef_cook_server_chop",
                "souchef_cook_server_plate",
                "souchef_plate_server_cook",
                "server_chop_souchef_cook"]
    final_record = []
    for i in np.arange(1, 7):
        cur_record = []
        for feature in features:
            if feature in ["chef_cook_souchef_chop", "chef_cook_server_plate"] and i > 2:
                cur_record.append(0)
            else:
                x = group1[i][feature]
                y = group2[i][feature]
                sample_len = min(x.shape[0], y.shape[0])
                cur_record.append(scipy.stats.wilcoxon(
                    x[:sample_len], y[:sample_len]
                )[1])
        final_record.append(cur_record)
    df = pd.DataFrame(final_record).T
    df = df.rename(columns={0:"Round 1", 1:"Round 2", 2:"Round 3", 3:"Round 4", 4:"Round 5", 5:"Round 6"})
    df = df.rename({0:"chef cook souchef chop", 1:"chef cook server plate", 2:"souchef chop server cook", 
                    3:"souchef cook server chop", 4:"souchef cook server plate", 5:"souchef plate server cook",
                   6: "server chop souchef cook"}, axis=0)
    df = df.reset_index()
    print(
        df.to_latex(index=False,
                    formatters={"name": str.upper},
                    float_format="{:.3f}".format)
    )
    return df

wilcoxin_ticks(s1_rounds_t, control_rounds_t)

\begin{tabular}{lrrrrrr}
\toprule
index & Round 1 & Round 2 & Round 3 & Round 4 & Round 5 & Round 6 \\
\midrule
chef cook souchef chop & 0.193 & 0.663 & 0.000 & 0.000 & 0.000 & 0.000 \\
chef cook server plate & 0.401 & 0.666 & 0.000 & 0.000 & 0.000 & 0.000 \\
souchef chop server cook & 0.715 & 0.122 & 0.000 & 0.000 & 0.000 & 0.009 \\
souchef cook server chop & 0.783 & 0.436 & 0.021 & 0.000 & 0.000 & 0.040 \\
souchef cook server plate & 0.050 & 0.880 & 0.852 & 0.172 & 0.700 & 0.229 \\
souchef plate server cook & 0.705 & 0.180 & 0.030 & 0.021 & 0.116 & 0.274 \\
server chop souchef cook & 0.644 & 0.366 & 0.364 & 0.035 & 0.141 & 0.117 \\
\bottomrule
\end{tabular}



/opt/homebrew/lib/python3.11/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,index,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6
0,chef cook souchef chop,0.193209,0.662923,0.000000,0.000000e+00,0.000000,0.000000
1,chef cook server plate,0.400781,0.666218,0.000000,0.000000e+00,0.000000,0.000000
2,souchef chop server cook,0.715001,0.122309,0.000012,5.633091e-07,0.000002,0.008952
3,souchef cook server chop,0.782594,0.436286,0.021376,3.202188e-07,0.000087,0.039926
4,souchef cook server plate,0.050481,0.880279,0.852294,1.722103e-01,0.700016,0.229085
5,souchef plate server cook,0.705457,0.179712,0.029940,2.135676e-02,0.116296,0.273830
6,server chop souchef cook,0.644223,0.366247,0.364403,3.496094e-02,0.141157,0.116998


### ttest

In [81]:
def ttest_ticks(group1, group2):
# group1 = s1_rounds_t
# group2 = control_rounds_t

    features = ["chef_cook_souchef_chop",
                "chef_cook_server_plate",
                "souchef_chop_server_cook",
                "souchef_cook_server_chop",
                "souchef_cook_server_plate",
                "souchef_plate_server_cook",
                "server_chop_souchef_cook"]
    final_record = []
    for i in np.arange(1, 7):
        cur_record = []
        for feature in features:
            if feature in ["chef_cook_souchef_chop", "chef_cook_server_plate"] and i > 2:
                cur_record.append(0)
            else:
                x = group1[i][feature]
                y = group2[i][feature]
                sample_len = min(x.shape[0], y.shape[0])
                cur_record.append(scipy.stats.ttest_ind(
                    x[:sample_len], y[:sample_len]
                )[1])
        final_record.append(cur_record)
    df = pd.DataFrame(final_record).T
    df = df.rename(columns={0:"Round 1", 1:"Round 2", 2:"Round 3", 3:"Round 4", 4:"Round 5", 5:"Round 6"})
    df = df.rename({0:"chef cook souchef chop", 1:"chef cook server plate", 2:"souchef chop server cook", 
                    3:"souchef cook server chop", 4:"souchef cook server plate", 5:"souchef plate server cook",
                   6: "server chop souchef cook"}, axis=0)
    df = df.reset_index()
    print(
        df.to_latex(index=False,
                    formatters={"name": str.upper},
                    float_format="{:.3f}".format)
    )
    return df
df1 = ttest_ticks(t1_rounds_t, control_rounds_t)

\begin{tabular}{lrrrrrr}
\toprule
index & Round 1 & Round 2 & Round 3 & Round 4 & Round 5 & Round 6 \\
\midrule
chef cook souchef chop & 0.581 & 0.326 & 0.000 & 0.000 & 0.000 & 0.000 \\
chef cook server plate & 0.744 & 0.162 & 0.000 & 0.000 & 0.000 & 0.000 \\
souchef chop server cook & 0.596 & 0.439 & 0.000 & 0.000 & 0.000 & 0.000 \\
souchef cook server chop & 0.312 & 0.712 & 0.112 & 0.000 & 0.000 & 0.000 \\
souchef cook server plate & 0.049 & 0.691 & 0.376 & 0.013 & 0.017 & 0.001 \\
souchef plate server cook & 0.695 & 0.180 & 0.000 & 0.000 & 0.000 & 0.000 \\
server chop souchef cook & 0.551 & 0.278 & 1.000 & 0.216 & 0.067 & 0.099 \\
\bottomrule
\end{tabular}



In [82]:
df2 = ttest_ticks(s1_rounds_t, control_rounds_t)

\begin{tabular}{lrrrrrr}
\toprule
index & Round 1 & Round 2 & Round 3 & Round 4 & Round 5 & Round 6 \\
\midrule
chef cook souchef chop & 0.359 & 0.705 & 0.000 & 0.000 & 0.000 & 0.000 \\
chef cook server plate & 0.489 & 0.796 & 0.000 & 0.000 & 0.000 & 0.000 \\
souchef chop server cook & 0.659 & 0.103 & 0.000 & 0.000 & 0.000 & 0.008 \\
souchef cook server chop & 0.905 & 0.262 & 0.280 & 0.000 & 0.000 & 0.061 \\
souchef cook server plate & 0.062 & 0.951 & 0.856 & 0.368 & 0.374 & 0.131 \\
souchef plate server cook & 0.491 & 0.180 & 0.029 & 0.078 & 0.188 & 0.166 \\
server chop souchef cook & 0.636 & 0.361 & 0.448 & 0.078 & 0.275 & 0.113 \\
\bottomrule
\end{tabular}



In [92]:
for i in np.arange(1, df1.shape[1]):
    df.iloc[:, i] = df.iloc[:, i].astype(str)[:3]
    

ValueError: Must have equal len keys and value when setting with an iterable

In [96]:
df.iloc[:, 1].astype(str)

0     0.19320899111601875
1      0.4007812008134437
2      0.7150006546880893
3       0.782593714752721
4    0.050480769146395967
5      0.7054569861112734
6      0.6442226424008191
Name: Round 1, dtype: object

In [11]:
# class add_idle():
    
#     def __init__(self, df):
#         self.df = df
        
#         self.chef_idle = []
#         self.souchef_idle = []
#         self.server_idle = []
        
#         self.assignment = {
#             "chef":0,
#             "souchef":0,
#             ""
#         }
        
#     def fit(self):
#         for player in self.df["ResponseId"].unique():
#             player_log = self.df[self.df["ResponseId"] == player]
#             for i in np.arange(player_log.shape[0]):
#                 workerId = player_log.iloc[i, 6] 
#                 if workerId != 0:
#                     self._update_assignment(
#                         player_log.iloc[i, :] 
#                     )
#                 else:
#                     self._check_assignment(
#                         player_log.iloc[i, :] 
#                     )        
#             self.assignment = {}
    
#     def _update_assignment(row):
        
        
                
                    
                


        






# s1

In [213]:
import itertools
from tqdm import tqdm

# def find_string_combinations(arr):
#     combinations = list(itertools.combinations(arr, 2))
#     return combinations
def find_string_combinations(arr):
    combinations = list(itertools.permutations(arr, 2))
    return combinations

arr = ["chef_chop", "chef_cook", "chef_plate", 
       "souchef_chop", "souchef_cook", "souchef_plate",
      "server_chop", "server_cook", "server_plate"]

In [14]:
def frequency_dictionary_update(df, spans=3):

    steps = spans - 1

    df = df.join(pd.get_dummies(df["taskId"]))
    df = df.rename(columns={0:"idle_2", 1:"chop", 2:"cook", 3:"plate"})

    df = df.join(pd.get_dummies(df["workerId"]))
    warmup = df[df["round"] <= 2]
    warmup = warmup.rename(columns={0:"idle", 1:"chef", 2:"souchef", 3:"server"})
    start = df[df["round"] >= 3]
#     start = start.rename(columns={0:"idle", 1:"chef", 2:"server", 3:"souchef"})
    start = start.rename(columns={0:"idle", 1:"souchef", 2:"server", 3:"chef"})
    start = start.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,15]]
    df = pd.concat([warmup, start],axis=0)

    df["chef_chop"] = df["chef"] & df["chop"]
    df["chef_cook"] = df["chef"] & df["cook"]
    df["chef_plate"] = df["chef"] & df["plate"]

    df["souchef_chop"] = df["souchef"] & df["chop"]
    df["souchef_cook"] = df["souchef"] & df["cook"]
    df["souchef_plate"] = df["souchef"] & df["plate"]

    df["server_chop"] = df["server"] & df["chop"]
    df["server_cook"] = df["server"] & df["cook"]
    df["server_plate"] = df["server"] & df["plate"]

    arr = ["chef_chop", "chef_cook", "chef_plate", 
           "souchef_chop", "souchef_cook", "souchef_plate",
          "server_chop", "server_cook", "server_plate"]
#     arr = ["chef_chop", "chef_cook", "chef_plate","souchef_chop", "souchef_cook"]
    
    feature_panels = find_string_combinations(arr)

    storage_final = {}
    dat = df.copy()

    for feature_panel in tqdm(feature_panels):
        feature_a = feature_panel[0]
        feature_b = feature_panel[1]
        for i in np.arange(1,7):
            df = df[df["round"] == i]
            feature_panels = find_string_combinations(arr)
            storage = {}
            for responser in df["ResponseId"].unique():
                sub_df = df[df["ResponseId"] == responser]
                for t in np.sort(sub_df["tick"].unique())[:-steps]:
                    if np.sum(sub_df[sub_df["tick"] == t][feature_a]) > 0:
                        window = sub_df[(sub_df["tick"] >= t) & (sub_df["tick"] <= t+steps)]
                        if np.sum(window[feature_b]) > 0:
                            feature = feature_a + "_" + feature_b
                            if not storage.get(feature):
                                storage[feature] = np.array([1])
                            else: 
                                storage[feature] += 1
                                
            for k, v in storage.items():
                storage[k] = np.round(v / len(df["ResponseId"].unique()), 2)
            # update to major
            if not storage_final.get(i):
                storage_final[i] = storage
            else:
                for k, v in storage.items():
                    if not storage_final[i].get(k):
                        storage_final[i][k] = storage[k]
                    else:
                        storage_final[i][k] += storage[k]
            df = dat
    return storage_final

In [15]:
control_frequency = frequency_dictionary_update(control, 6)
s1_frequency = frequency_dictionary_update(s1, 6)
t1_frequency = frequency_dictionary_update(t1, 6)

100%|█████████████████████████████████████████████████████████████████████████████| 72/72 [42:01<00:00, 35.02s/it]


In [187]:
control_frequency[1]

{'chef_chop_chef_cook': array([1.63]),
 'chef_chop_chef_plate': array([0.03]),
 'chef_chop_souchef_chop': array([1.07]),
 'chef_chop_souchef_cook': array([1.03]),
 'chef_chop_souchef_plate': array([0.36]),
 'chef_chop_server_chop': array([0.89]),
 'chef_chop_server_cook': array([0.47]),
 'chef_chop_server_plate': array([0.48]),
 'chef_cook_chef_chop': array([0.23]),
 'chef_cook_chef_plate': array([0.45]),
 'chef_cook_souchef_chop': array([0.39]),
 'chef_cook_souchef_cook': array([1.29]),
 'chef_cook_souchef_plate': array([0.78]),
 'chef_cook_server_chop': array([0.37]),
 'chef_cook_server_cook': array([0.54]),
 'chef_cook_server_plate': array([2.06]),
 'chef_plate_chef_chop': array([0.03]),
 'chef_plate_chef_cook': array([0.02]),
 'chef_plate_souchef_chop': array([0.11]),
 'chef_plate_souchef_cook': array([0.11]),
 'chef_plate_souchef_plate': array([0.32]),
 'chef_plate_server_chop': array([0.02]),
 'chef_plate_server_cook': array([0.01]),
 'chef_plate_server_plate': array([0.04]),
 's

In [16]:
s1_frequency[1]

{'chef_chop_chef_cook': array([1.53]),
 'chef_chop_chef_plate': array([0.07]),
 'chef_chop_souchef_chop': array([1.09]),
 'chef_chop_souchef_cook': array([1.1]),
 'chef_chop_souchef_plate': array([0.34]),
 'chef_chop_server_chop': array([0.9]),
 'chef_chop_server_cook': array([0.47]),
 'chef_chop_server_plate': array([0.45]),
 'chef_cook_chef_chop': array([0.25]),
 'chef_cook_chef_plate': array([0.4]),
 'chef_cook_souchef_chop': array([0.49]),
 'chef_cook_souchef_cook': array([1.2]),
 'chef_cook_souchef_plate': array([0.63]),
 'chef_cook_server_chop': array([0.37]),
 'chef_cook_server_cook': array([0.5]),
 'chef_cook_server_plate': array([2.18]),
 'chef_plate_chef_chop': array([0.01]),
 'chef_plate_chef_cook': array([0.01]),
 'chef_plate_souchef_chop': array([0.05]),
 'chef_plate_souchef_cook': array([0.11]),
 'chef_plate_souchef_plate': array([0.29]),
 'chef_plate_server_cook': array([0.]),
 'chef_plate_server_plate': array([0.07]),
 'souchef_chop_chef_chop': array([1.12]),
 'souchef_

In [17]:
# def frequency_dictionary(df, spans=3):
#     worker_id = [1,2,3]
#     tasks_id = [1,2,3]
#     steps = spans - 1

#     df = df.join(pd.get_dummies(df["taskId"]))
#     df = df.rename(columns={0:"idle_2", 1:"chop", 2:"cook", 3:"plate"})

#     df = df.join(pd.get_dummies(df["workerId"]))
#     warmup = df[df["round"] <= 2]
#     warmup = warmup.rename(columns={0:"idle", 1:"chef", 2:"souchef", 3:"server"})
#     start = df[df["round"] >= 3]
#     start = start.rename(columns={0:"idle", 1:"chef", 2:"server", 3:"souchef"})
#     start = start.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,15]]
#     df = pd.concat([warmup, start],axis=0)

#     df["chef_chop"] = df["chef"] & df["chop"]
#     df["chef_cook"] = df["chef"] & df["cook"]
#     df["chef_plate"] = df["chef"] & df["plate"]

#     df["souchef_chop"] = df["souchef"] & df["chop"]
#     df["souchef_cook"] = df["souchef"] & df["cook"]
#     df["souchef_plate"] = df["souchef"] & df["plate"]

#     df["server_chop"] = df["server"] & df["chop"]
#     df["server_cook"] = df["server"] & df["cook"]
#     df["server_plate"] = df["server"] & df["plate"]

#     arr = ["chef_chop", "chef_cook", "chef_plate", 
#            "souchef_chop", "souchef_cook", "souchef_plate",
#           "server_chop", "server_cook", "server_plate", "idle"]

#     storage_final = {}
#     dat = df.copy()
#     for i in tqdm(np.arange(1,7)):
#         df = df[df["round"] == i]
#         feature_panels = find_string_combinations(arr)
#         storage = {}
#         for responser in df["ResponseId"].unique():
#             sub_df = df[df["ResponseId"] == responser]
#             for t in np.sort(sub_df["tick"].unique())[:-steps]:
#                 sub_df = sub_df[(sub_df["tick"] >= t) & (sub_df["tick"] <= t+steps)]
#                 for feature_panel in feature_panels:
#                     feature_a = feature_panel[0]
#                     feature_b = feature_panel[1]
#                     if np.sum(sub_df[feature_a]) > 0 and np.sum(sub_df[feature_b]) > 0:
#                         feature = feature_a + "_" + feature_b
#                         count = min(np.sum(sub_df[feature_a]), np.sum(sub_df[feature_b]))
#                         if not storage.get(feature):
#                             storage[feature] = np.array([count])
#                         else: 
#                             storage[feature] += count
#         for k, v in storage.items():
#             storage[k] = np.round(v / len(df["ResponseId"].unique()), 2)
#         df = dat
#         storage_final[i] = storage
#     return storage_final      

In [18]:
# s1_frequency = frequency_dictionary(s1, 6)
# t1_frequency = frequency_dictionary(t1, 6)

In [21]:
def dic_to_df(dic):
    try:
        round1 = pd.DataFrame(dic[1]).T.reset_index()
    except:
        pdb.set_trace()
    round2 = pd.DataFrame(dic[2]).T.rename(columns={0:1}).reset_index()
    df = pd.merge(round1,round2, how='outer')

    round3 = pd.DataFrame(dic[3]).T.rename(columns={0:2}).reset_index()
    df = pd.merge(df,round3, how='outer')

    round4 = pd.DataFrame(dic[4]).T.rename(columns={0:3}).reset_index()
    df = pd.merge(df,round4, how='outer')

    round5 = pd.DataFrame(dic[5]).T.rename(columns={0:4}).reset_index()
    df = pd.merge(df,round5, how='outer')

    round6 = pd.DataFrame(dic[6]).T.rename(columns={0:5}).reset_index()
    df = pd.merge(df,round6, how='outer')

    df = df.fillna(0).rename(columns = {"index":"strategy", 
                                  0:"round 1",
                                  1:"round 2",
                                  2:"round 3",
                                  3:"round 4",
                                  4:"round 5",
                                  5:"round 6"})
    return df

In [66]:
pd.merge(final, dic_to_df(control_frequency), on="strategy")

,strategy,round 1_x,round 2_x,round 3_x,round 4_x,round 5_x,round 6_x,round 1_y,round 2_y,round 3_y,round 4_y,round 5_y,round 6_y,round 1,round 2,round 3,round 4,round 5,round 6
0,chef_chop_chef_cook,1.53,1.50,0.00,0.00,0.00,0.00,1.58,1.51,0.00,0.00,0.00,0.00,1.63,1.58,0.00,0.00,0.00,0.00
1,chef_chop_chef_plate,0.07,0.01,0.00,0.00,0.00,0.00,0.02,0.03,0.00,0.00,0.00,0.00,0.03,0.01,0.00,0.00,0.00,0.00
2,chef_chop_souchef_chop,1.09,1.00,0.00,0.00,0.00,0.00,1.05,0.99,0.00,0.00,0.00,0.00,1.07,1.07,0.00,0.00,0.00,0.00
3,chef_chop_souchef_cook,1.10,0.93,0.00,0.00,0.00,0.00,1.10,0.94,0.00,0.00,0.00,0.00,1.03,0.90,0.00,0.00,0.00,0.00
4,chef_chop_souchef_plate,0.34,0.19,0.00,0.00,0.00,0.00,0.36,0.20,0.00,0.00,0.00,0.00,0.36,0.18,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,server_plate_souchef_plate,0.59,0.66,1.00,0.98,1.00,0.81,0.58,0.59,0.85,0.88,0.78,0.81,0.70,0.72,0.87,0.73,0.75,0.73
66,server_plate_server_chop,0.25,0.40,0.81,1.09,1.01,0.84,0.31,0.41,0.67,0.91,1.05,0.84,0.28,0.42,0.63,0.76,0.67,0.78
67,server_cook_souchef_chop,0.00,0.02,0.11,0.17,0.11,0.12,0.00,0.02,0.25,0.30,0.34,0.38,0.00,0.00,0.05,0.09,0.11,0.10
68,server_cook_server_chop,0.00,0.00,0.05,0.06,0.04,0.03,0.00,0.00,0.07,0.06,0.05,0.07,0.00,0.00,0.02,0.05,0.02,0.02


In [201]:
final = pd.merge(dic_to_df(s1_frequency), dic_to_df(t1_frequency), on="strategy")
final = pd.merge(final, dic_to_df(control_frequency), on="strategy")
s1_final = final.iloc[:, 0:7].rename(columns={
    "round 1_x":"round 1","round 2_x":"round 2","round 3_x":"round 3",
    "round 4_x":"round 4","round 5_x":"round 5","round 6_x":"round 6",
})
t1_final = final.iloc[:, [0,7,8,9,10,11,12]].rename(columns={
    "round 1_y":"round 1","round 2_y":"round 2","round 3_y":"round 3",
    "round 4_y":"round 4","round 5_y":"round 5","round 6_y":"round 6",
})
control_final = final.iloc[:, [0, 13,14,15,16,17,18]].rename(columns={
    "round 1_z":"round 1","round 2_z":"round 2","round 3_z":"round 3",
    "round 4_z":"round 4","round 5_z":"round 5","round 6_z":"round 6",
})
dif = s1_final.iloc[:,1:] - t1_final.iloc[:,1:]
# dif
dif_st = s1_final.iloc[:,1:] - t1_final.iloc[:,1:]
dif_ct = control_final.iloc[:,1:] - t1_final.iloc[:,1:]
dif_cs = control_final.iloc[:,1:] - s1_final.iloc[:,1:]

In [202]:
rows = [10, 15, 29, 36, 38, 45, 51]

In [203]:
dif_st.iloc[rows,:]

,round 1,round 2,round 3,round 4,round 5,round 6
10,0.05,-0.14,0.00,0.00,0.00,0.00
15,0.08,-0.29,0.00,0.00,0.00,0.00
29,0.01,0.03,-0.16,-0.22,-0.49,-0.64
36,0.03,0.03,0.06,0.16,0.23,0.41
38,-0.06,-0.03,-0.17,0.32,0.35,0.28
45,0.00,0.00,-0.06,-0.19,-0.23,-0.25
51,-0.04,0.09,0.07,0.32,0.31,0.31


In [205]:
dif_ct.iloc[rows,:]

,round 1,round 2,round 3,round 4,round 5,round 6
10,-0.05,-0.11,0.00,0.00,0.00,0.00
15,-0.04,-0.24,0.00,0.00,0.00,0.00
29,0.02,-0.03,-0.59,-0.73,-0.91,-0.88
36,0.03,-0.01,0.19,0.60,0.62,0.57
38,-0.18,-0.03,-0.22,0.52,0.55,0.56
45,0.01,-0.01,-0.15,-0.27,-0.29,-0.31
51,-0.02,0.05,0.00,0.14,0.20,0.17


In [206]:
dif_cs.iloc[rows,:]

,round 1,round 2,round 3,round 4,round 5,round 6
10,-0.10,0.03,0.00,0.00,0.00,0.00
15,-0.12,0.05,0.00,0.00,0.00,0.00
29,0.01,-0.06,-0.43,-0.51,-0.42,-0.24
36,0.00,-0.04,0.13,0.44,0.39,0.16
38,-0.12,0.00,-0.05,0.20,0.20,0.28
45,0.01,-0.01,-0.09,-0.08,-0.06,-0.06
51,0.02,-0.04,-0.07,-0.18,-0.11,-0.14


In [88]:
# dif[30:]

In [75]:
s1_final.iloc[rows,:]

,strategy,round 1,round 2,round 3,round 4,round 5,round 6
10,chef_cook_souchef_chop,0.49,0.68,0.00,0.00,0.00,0.00
15,chef_cook_server_plate,2.18,2.93,0.00,0.00,0.00,0.00
29,souchef_chop_server_cook,0.46,0.19,0.75,1.01,0.93,0.89
36,souchef_cook_server_chop,0.10,0.16,1.29,0.97,0.91,0.98
38,souchef_cook_server_plate,0.69,0.81,1.71,1.67,1.62,1.40
45,souchef_plate_server_cook,0.01,0.01,0.12,0.14,0.13,0.12
51,server_chop_souchef_cook,0.87,0.68,1.64,1.71,1.60,1.57


In [83]:
t1_final.iloc[rows,:]

,strategy,round 1,round 2,round 3,round 4,round 5,round 6
10,chef_cook_souchef_chop,0.44,0.82,0.00,0.00,0.00,0.00
15,chef_cook_server_plate,2.10,3.22,0.00,0.00,0.00,0.00
29,souchef_chop_server_cook,0.45,0.16,0.91,1.23,1.42,1.53
36,souchef_cook_server_chop,0.07,0.13,1.23,0.81,0.68,0.57
38,souchef_cook_server_plate,0.75,0.84,1.88,1.35,1.27,1.12
45,souchef_plate_server_cook,0.01,0.01,0.18,0.33,0.36,0.37
51,server_chop_souchef_cook,0.91,0.59,1.57,1.39,1.29,1.26


In [98]:
rows = [10, 15, 29, 36, 38, 45, 51]

In [100]:
control_final.iloc[rows,1:]

,round 1,round 2,round 3,round 4,round 5,round 6
10,0.39,0.71,0.00,0.00,0.00,0.00
15,2.06,2.98,0.00,0.00,0.00,0.00
29,0.47,0.13,0.32,0.50,0.51,0.65
36,0.10,0.12,1.42,1.41,1.30,1.14
38,0.57,0.81,1.66,1.87,1.82,1.68
45,0.02,0.00,0.03,0.06,0.07,0.06
51,0.89,0.64,1.57,1.53,1.49,1.43


In [102]:
dif_cs.iloc[rows,:]

,round 1,round 2,round 3,round 4,round 5,round 6,strategy
10,-0.10,0.03,0.00,0.00,0.00,0.00,NaN
15,-0.12,0.05,0.00,0.00,0.00,0.00,NaN
29,0.01,-0.06,-0.43,-0.51,-0.42,-0.24,NaN
36,0.00,-0.04,0.13,0.44,0.39,0.16,NaN
38,-0.12,0.00,-0.05,0.20,0.20,0.28,NaN
45,0.01,-0.01,-0.09,-0.08,-0.06,-0.06,NaN
51,0.02,-0.04,-0.07,-0.18,-0.11,-0.14,NaN


In [52]:
# dif.iloc[[7,8,4,14,15,16,20],:]
dif.iloc[rows,:]

,round 1,round 2,round 3,round 4,round 5,round 6
10,0.05,-0.14,0.00,0.00,0.00,0.00
15,0.08,-0.29,0.00,0.00,0.00,0.00
29,0.01,0.03,-0.16,-0.22,-0.49,-0.64
36,0.03,0.03,0.06,0.16,0.23,0.41
38,-0.06,-0.03,-0.17,0.32,0.35,0.28
45,0.00,0.00,-0.06,-0.19,-0.23,-0.25
51,-0.04,0.09,0.07,0.32,0.31,0.31
65,0.01,0.07,0.15,0.10,0.22,0.00


In [53]:
dif_ct.iloc[rows,:]

,round 1,round 2,round 3,round 4,round 5,round 6
10,-0.05,-0.11,0.00,0.00,0.00,0.00
15,-0.04,-0.24,0.00,0.00,0.00,0.00
29,0.02,-0.03,-0.59,-0.73,-0.91,-0.88
36,0.03,-0.01,0.19,0.60,0.62,0.57
38,-0.18,-0.03,-0.22,0.52,0.55,0.56
45,0.01,-0.01,-0.15,-0.27,-0.29,-0.31
51,-0.02,0.05,0.00,0.14,0.20,0.17
65,0.12,0.13,0.02,-0.15,-0.03,-0.08


In [70]:
control_final.iloc[rows,:]

,strategy,round 1,round 2,round 3,round 4,round 5,round 6
10,chef_cook_souchef_chop,0.39,0.71,0.00,0.00,0.00,0.00
15,chef_cook_server_plate,2.06,2.98,0.00,0.00,0.00,0.00
29,souchef_chop_server_cook,0.47,0.13,0.32,0.50,0.51,0.65
36,souchef_cook_server_chop,0.10,0.12,1.42,1.41,1.30,1.14
38,souchef_cook_server_plate,0.57,0.81,1.66,1.87,1.82,1.68
45,souchef_plate_server_cook,0.02,0.00,0.03,0.06,0.07,0.06
51,server_chop_souchef_cook,0.89,0.64,1.57,1.53,1.49,1.43
65,server_plate_souchef_plate,0.70,0.72,0.87,0.73,0.75,0.73


In [61]:
# s1_final.iloc[[7,8,4,14,15,16,20],:]
s1_final.iloc[rows,:]

,strategy,round 1,round 2,round 3,round 4,round 5,round 6
10,chef_cook_souchef_chop,0.49,0.68,0.00,0.00,0.00,0.00
15,chef_cook_server_plate,2.18,2.93,0.00,0.00,0.00,0.00
29,souchef_chop_server_cook,0.46,0.19,0.75,1.01,0.93,0.89
36,souchef_cook_server_chop,0.10,0.16,1.29,0.97,0.91,0.98
38,souchef_cook_server_plate,0.69,0.81,1.71,1.67,1.62,1.40
45,souchef_plate_server_cook,0.01,0.01,0.12,0.14,0.13,0.12
51,server_chop_souchef_cook,0.87,0.68,1.64,1.71,1.60,1.57
65,server_plate_souchef_plate,0.59,0.66,1.00,0.98,1.00,0.81


In [72]:
# t1_final.iloc[[7,8,4,14,15,16,20],:]
t1_final.iloc[rows,:]

,strategy,round 1,round 2,round 3,round 4,round 5,round 6
10,chef_cook_souchef_chop,0.44,0.82,0.00,0.00,0.00,0.00
11,chef_cook_souchef_cook,1.16,1.09,0.00,0.00,0.00,0.00
15,chef_cook_server_plate,2.10,3.22,0.00,0.00,0.00,0.00
29,souchef_chop_server_cook,0.45,0.16,0.91,1.23,1.42,1.53
36,souchef_cook_server_chop,0.07,0.13,1.23,0.81,0.68,0.57
38,souchef_cook_server_plate,0.75,0.84,1.88,1.35,1.27,1.12
45,souchef_plate_server_cook,0.01,0.01,0.18,0.33,0.36,0.37
51,server_chop_souchef_cook,0.91,0.59,1.57,1.39,1.29,1.26
65,server_plate_souchef_cook,0.13,0.18,1.51,1.13,0.99,0.90
68,server_cook_souchef_chop,0.00,0.02,0.25,0.30,0.34,0.38


In [ ]:
for featureA_featureB, find when featureA is taken place (t1, t2, ... tn), for each t_i in (t_1 ... t_n), check if featureB exists within the window (t_i, t_i+5). Sum all players and all corresponding boolen value (0 or 1), and divide by the number of players.